In [2]:
#invoke preprocessing methods
%run preprocessing.ipynb

### Calculate TF-IDF using Scikit-Learn

In [19]:
import os
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

'''
    Processing the raw json documents
    params: raw json documents folder
    notice: run preprocessing.ipynb first, to make sure that preprocessing methods is equiped
'''

def scikit_tf_idf_rawDocument(raw_doc_path):
    dictionary = set()
    documents_list = []
    files_list = []
    for root, dirs, files in os.walk(raw_doc_path):
        for f in files:
            with open(root+'/'+f) as fs:
                files_list.append(f)
                print('.', end='')
                words_list = ''
                try:
                    for line in fs:
                        #loads json file, preprocess the content
                        obj = json.loads(line)
                        #only extract useful information [type--paragraph--content]
                        textType = obj['type']
                        if textType == 'paragraph':
                            words = preprocessing_spacy(obj['content'])
                            for word in words:
                                dictionary.add(word)
                                words_list += word + ' '
                except:
                    print ('Error while processing file: ', f)
            documents_list.append(words_list)
#     return dictionary, documents_list, files_list

#     dictionary, documents_list, files_list = documentProcessing()
    sklearn_vector = TfidfVectorizer(vocabulary=dictionary)
    sklearn_tf_idf = sklearn_vector.fit_transform(documents_list)
    return sklearn_vector, sklearn_tf_idf, files_list

In [20]:
def sklearn_find_top_k_words(sklearn_vector, tf_idf, k, filenames):
    document_freq_words = {}
    corpus = sklearn_vector.get_feature_names()
    for i in range(tf_idf.shape[1]):
        freq_words = []
        #get indices of k-maximum values in numpy column
        index = np.argpartition(tf_idf[:, i], -k)[-k:]
        for ind in index:
            freq_words.append(corpus[ind])
        document_freq_words[filenames[i].split('.')[0]] = freq_words
        df = pd.DataFrame(document_freq_words)
    return df

### Find most frequent 10 keywords using Scikit-Learn tf-idf library

In [9]:
%%time
#path for raw json documents
raw_doc_path = '/home/bit/ma0/LabShare/data/three-companies/json'
sklearn_vector, sklearn_tf_idf, files_list = scikit_tf_idf_rawDocument(raw_doc_path)
sk_tf_idf = np.transpose(sklearn_tf_idf)
# sklearn_find_top_k_words(sklearn_vector, sk_tf_idf.toarray(), 15, files_list)

.......................CPU times: user 10min 17s, sys: 29.4 s, total: 10min 46s
Wall time: 1min 20s


In [10]:
# sklearn_find_top_k_words(sklearn_vector, sk_tf_idf.toarray(), 15, files_list)

### Calculate TF-IDF on preprocessed documents, to find most frequent words

In [46]:
import pickle
def scikit_tf_idf_PrerocessedDoc(processed_docu_path, document_limit):
    dictionary = set()
    documents_list = []
    files_list = []
    for root, dirs, files in os.walk(processed_docu_path):
        for f in files:
            document_limit -= 1
            files_list.append(f)
            print('.', end='')
            words_list = ''
            with open(root+'/'+f, 'rb') as fr:
                try:
                    words = pickle.load(fr)
                    for word in words:
                        dictionary.add(word)
                        words_list += word + ' '
                except:
                    print ('Error while processing file: ', f)
            
            documents_list.append(words_list)
#             print(len(dictionary))
            if document_limit == 0:
                break
#     print(documents_list)
    sklearn_vector = TfidfVectorizer(vocabulary=dictionary)
    sklearn_tf_idf = sklearn_vector.fit_transform(documents_list)
    return sklearn_vector, sklearn_tf_idf, files_list

In [61]:
%%time
#path for raw json documents
# processed_doc_path = './LabShare/data/chui_ma/spacy_corpus'
# new_sklearn_vector, new_sklearn_tf_idf, new_files_list = scikit_tf_idf_PrerocessedDoc(processed_doc_path, 100)
# new_sk_tf_idf = np.transpose(new_sklearn_tf_idf)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.7 µs


In [64]:
# %%time
# sklearn_find_top_k_words(new_sklearn_vector, new_sk_tf_idf.toarray(), 15, new_files_list)

In [57]:
# path = './LabShare/data/chui_ma/spacy_corpus/OSRAM-QuarterlyReport-2016-Q3_corpus.txt'
# with open(path, 'rb') as f:
#     s = pickle.load(f)
# s

In [ ]:
|